## Imports:

In [ ]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [ ]:
dir_path = "/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Raw_Recordings"

In [ ]:
mr_ = np.load(dir_path + "/trimmed_t_mr.npy", allow_pickle= True)
ms_ = np.load(dir_path + "/trimmed_t_ms.npy", allow_pickle= True)
as_ = np.load(dir_path + "/trimmed_t_as.npy", allow_pickle= True)
ar_ = np.load(dir_path + "/trimmed_t_ar.npy", allow_pickle= True)

In [ ]:
mr_.shape

In [ ]:
d = obspy.core.trace.Trace(mr_[0][0])
type(d)

In [ ]:
def converter(array):
    x,y,z = [],[],[]
    for i in range(array.shape[1]):
        x.append(obspy.core.trace.Trace(array[0][i]))
        y.append(obspy.core.trace.Trace(array[1][i]))
        z.append(obspy.core.trace.Trace(array[2][i]))
        
        x[i].stats.sampling_rate = 256
        y[i].stats.sampling_rate = 256
        z[i].stats.sampling_rate = 256
    return [x,y,z]

In [ ]:
mr_ = converter(mr_)
ms_ = converter(ms_)
as_ = converter(as_)
ar_ = converter(ar_)

In [ ]:
#sanity check
for i in range(len(ar_[1])):
    print(ar_[0][0] == ar_[0][i])

## comparison of two dataset:

Now we will to try visualize spectograms. So far we have used Obspy to visualize SCG signals. However, with obspy we are not able to plot spectograms side by side. This is important for us because we want to see spectograms of x,y and z components simultaneously for both healty and diseased people. So, from now on I will refer to those plots, for 3 components of healty and diseased (in total six plot, as "PAIR".


I have mentioned that Obpsy has a problem in side by side plotting. So I changed it source code. I have added and deleted some part to operate according to my need. ???????????

Thats why from now on we are going to use **output_spectogram** function to get values needed to plot spectogram.

These are the imports for output_spectogram function to work.

In [ ]:
from obspy.imaging.cm import obspy_sequential
import math
from matplotlib import mlab
from matplotlib.colors import Normalize

In [ ]:
def _nearest_pow_2(x):
    a = math.pow(2, math.ceil(np.log2(x)))
    b = math.pow(2, math.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b

In [ ]:
def output_spectrogram(data, samp_rate, per_lap=0.9, wlen=None, log=False,
                outfile=None, fmt=None, axes=None, dbscale=False,
                mult=8.0, cmap=obspy_sequential, zorder=None, title=None,
                show=True, clip=[0.0, 1.0]):
 
    import matplotlib.pyplot as plt
    
    # enforce float for samp_rate
    samp_rate = float(samp_rate)

    # set wlen from samp_rate if not specified otherwise
    if not wlen:
        wlen = samp_rate / 100.

    npts = len(data)
    # nfft needs to be an integer, otherwise a deprecation will be raised
    # XXX add condition for too many windows => calculation takes for ever
    nfft = int(_nearest_pow_2(wlen * samp_rate))
    if nfft > npts:
        nfft = int(_nearest_pow_2(npts / 8.0))

    if mult is not None:
        mult = int(_nearest_pow_2(mult))
        mult = mult * nfft
    nlap = int(nfft * float(per_lap))

    data = data - np.array(data).mean()
    end = npts / samp_rate

    # Here we call not plt.specgram as this already produces a plot
    # matplotlib.mlab.specgram should be faster as it computes only the
    # arrays
    # XXX mlab.specgram uses fft, would be better and faster use rfft
    specgram, freq, time = mlab.specgram(data, Fs=samp_rate, NFFT=nfft,
                                         pad_to=mult, noverlap=nlap, mode = 'psd')

    # db scale and remove zero/offset for amplitude
    if dbscale:
        specgram = 10 * np.log10(specgram[1:, :])
    else:
        specgram = np.sqrt(specgram[1:, :])
    freq = freq[1:]
    
    vmin, vmax = clip
    if vmin < 0 or vmax > 1 or vmin >= vmax:
        msg = "Invalid parameters for clip option."
        raise ValueError(msg)
    _range = float(specgram.max() - specgram.min())
    vmin = specgram.min() + vmin * _range
    vmax = specgram.min() + vmax * _range
    norm = Normalize(vmin, vmax, clip=True)

    # calculate half bin width
    halfbin_time = (time[1] - time[0]) / 2.0
    halfbin_freq = (freq[1] - freq[0]) / 2.0

    # argument None is not allowed for kwargs on matplotlib python 3.3
    kwargs = {k: v for k, v in (('cmap', cmap), ('zorder', zorder))
              if v is not None}

    if log:
        # pcolor expects one bin more at the right end
        freq = np.concatenate((freq, [freq[-1] + 2 * halfbin_freq]))
        time = np.concatenate((time, [time[-1] + 2 * halfbin_time]))
        # center bin
        time -= halfbin_time
        freq -= halfbin_freq
        # Log scaling for frequency values (y-axis)
        ax.set_yscale('log')
        # Plot times
        ax.pcolormesh(time, freq, specgram, norm=norm, **kwargs)
    else:
        # this method is much much faster!
        specgram = np.flipud(specgram)
        # center bin
        extent = (time[0] - halfbin_time, time[-1] + halfbin_time,
                  freq[0] - halfbin_freq, freq[-1] + halfbin_freq)
    return [specgram, extent, end]

    



## Plotting spectogram:

f is the frequency array, containing the frequencies of every band of the fft. Which can be used as the labels for a graph

t is the time array, containing the time at which this FFT was made relative to the source signal. Again can be used for labels.

The Sxx array contains the amplitudes and is a 2d array whose shape is the length of f by the length of t.


 The purpose of a spectogram is to take the FFT of small, equal-sized time chunks. This produces a 2D fourier transform where the X axis is the start time of the time chunk and the Y axis is the energy (or power, etc.) in each frequency in that time chunk. This allows you to see how the frequency components change over time.
 
  A spectrogram is a representation of frequency over time with the addition of amplitude as a third dimension, denoting the intensity or volume of the signal at a frequency and a time.

In [ ]:
specgram, extent, end =  output_spectrogram(ms_[0][56].data, 256)

#### extent = (left, right, bottom, top), optional

In [ ]:
extent

In [ ]:
end

In [ ]:
specgram.shape #[freq, time]

In [ ]:
specgram[0:2048,0].mean() # average of frequency values at a time

In [ ]:
specgram[0:2048,8].shape

###  MS

In [ ]:
for i in range(len(ms_[0])):
    
        MS_s_x, MS_e_x, MS_end_x = output_spectrogram(ms_[0][i].data, 256)
        MS_s_y, MS_e_y, MS_end_y = output_spectrogram(ms_[1][i].data, 256)
        MS_s_z, MS_e_z, MS_end_z = output_spectrogram(ms_[2][i].data, 256)
        
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (18,4))

        ax1.imshow(MS_s_x, interpolation="nearest", extent= MS_e_x, vmin =0)
        ax1.axis('tight')
        ax1.set_xlim(0, MS_end_x)
        ax1.title.set_text('x comp. of MS subject {} SCG'.format(i))
        ax1.grid(False)
        fig.colorbar(ax1.images[0], ax = ax1)
       
        
        ax2.imshow(MS_s_y, interpolation="nearest", extent= MS_e_y, vmin = 0)
        ax2.axis('tight')
        ax2.set_xlim(0, MS_end_y)
        ax2.title.set_text('y comp. of MS subject {} SCG'.format(i))
        ax2.grid(False)
        fig.colorbar(ax2.images[0], ax = ax2)
        
        ax3.imshow(MS_s_z, interpolation="nearest", extent= MS_e_z, vmin =0)
        ax3.axis('tight')
        ax3.set_xlim(0, MS_end_z)
        ax3.title.set_text('z comp. of MS subject {} SCG'.format(i))
        ax3.grid(False)
        fig.colorbar(ax3.images[0], ax = ax3)
        
        #0.4

## MR

In [ ]:
for i in range(len(mr_[0])):
        
        MR_s_x, MR_e_x, MR_end_x = output_spectrogram(mr_[0][i].data, mr_[0][i].stats.sampling_rate)
        MR_s_y, MR_e_y, MR_end_y = output_spectrogram(mr_[1][i].data, mr_[1][i].stats.sampling_rate)
        MR_s_z, MR_e_z, MR_end_z = output_spectrogram(mr_[2][i].data, mr_[2][i].stats.sampling_rate)

        fig, (ax4, ax5, ax6) = plt.subplots(1, 3, figsize = (18,4))

        ax4.imshow(MR_s_x, interpolation="nearest", extent= MR_e_x, vmin = 0)
        ax4.axis('tight')
        ax4.set_xlim(0, MR_end_x)
        ax4.title.set_text('x comp. of MR subject {} SCG '.format(i))
        ax4.grid(False)
        fig.colorbar(ax4.images[0], ax = ax4)
        
        ax5.imshow(MR_s_y, interpolation="nearest", extent= MR_e_y, vmin = 0)
        ax5.axis('tight')
        ax5.set_xlim(0, MR_end_y)
        ax5.title.set_text('y comp. of MR subject {} SCG '.format(i))
        ax5.grid(False)
        fig.colorbar(ax5.images[0], ax = ax5)
    
        ax6.imshow(MR_s_z, interpolation="nearest", extent= MR_e_z, vmin = 0)
        ax6.axis('tight')
        ax6.set_xlim(0, MR_end_z)
        ax6.title.set_text('z comp. of MR subject {} SCG '.format(i))
        ax6.grid(False)
        fig.colorbar(ax6.images[0], ax = ax6)
        ### 335-334 1 bandında outlier ?
        #0.4-0.5

###  AS

In [ ]:
for i in range(len(as_[0])):
        
        AS_s_x, AS_e_x, AS_end_x = output_spectrogram(as_[0][i].data, as_[0][i].stats.sampling_rate)
        AS_s_y, AS_e_y, AS_end_y = output_spectrogram(as_[1][i].data, as_[1][i].stats.sampling_rate)
        AS_s_z, AS_e_z, AS_end_z = output_spectrogram(as_[2][i].data, as_[2][i].stats.sampling_rate)
        
        fig, (ax4, ax5, ax6) = plt.subplots(1, 3,  figsize=(18,4))

        ax4.imshow(AS_s_x, interpolation="nearest", extent= AS_e_x, vmin = 0)
        ax4.axis('tight')
        ax4.set_xlim(0, AS_end_x)
        ax4.title.set_text('x comp. of AS subject {} SCG '.format(i))
        ax4.grid(False)
        fig.colorbar(ax4.images[0], ax = ax4)
        
        ax5.imshow(AS_s_y, interpolation="nearest", extent= AS_e_y, vmin = 0)
        ax5.axis('tight')
        ax5.set_xlim(0, AS_end_y)
        ax5.title.set_text('y comp. of AS subject {} SCG '.format(i))
        ax5.grid(False)
        fig.colorbar(ax5.images[0], ax = ax5)
    
        ax6.imshow(AS_s_z, interpolation="nearest", extent= AS_e_z, vmin = 0)
        ax6.axis('tight')
        ax6.set_xlim(0, AS_end_z)
        ax6.title.set_text('z comp. of AS subject {} SCG '.format(i))
        ax6.grid(False)
        fig.colorbar(ax6.images[0], ax = ax6)
        #one 0.8 outlier_
        #0.4

###  AR

In [ ]:
for i in range(len(ar_[0])):
        
        AR_s_x, AR_e_x, AR_end_x = output_spectrogram(ar_[0][i].data, ar_[0][i].stats.sampling_rate)
        AR_s_y, AR_e_y, AR_end_y = output_spectrogram(ar_[1][i].data, ar_[1][i].stats.sampling_rate)
        AR_s_z, AR_e_z, AR_end_z = output_spectrogram(ar_[2][i].data, ar_[2][i].stats.sampling_rate)
        
        fig, (ax4, ax5, ax6) = plt.subplots(1, 3,  figsize=(18,4))

        ax4.imshow(AR_s_x, interpolation="nearest", extent= AR_e_x, vmin = 0)
        ax4.axis('tight')
        ax4.set_xlim(0, AR_end_x)
        ax4.title.set_text('x comp. of AR subject {} SCG '.format(i))
        ax4.grid(False)
        fig.colorbar(ax4.images[0], ax = ax4)
        
        ax5.imshow(AR_s_y, interpolation="nearest", extent= AR_e_y, vmin = 0)
        ax5.axis('tight')
        ax5.set_xlim(0, AR_end_y)
        ax5.title.set_text('y comp. of AR subject {} SCG '.format(i))
        ax5.grid(False)
        fig.colorbar(ax5.images[0], ax = ax5)
    
        ax6.imshow(AR_s_z, interpolation="nearest", extent= AR_e_z, vmin = 0)
        ax6.axis('tight')
        ax6.set_xlim(0, AR_end_z)
        ax6.title.set_text('z comp. of AR subject {} SCG '.format(i))
        ax6.grid(False)
        fig.colorbar(ax6.images[0], ax = ax6)
        #2 ?
        # bir tane 12 var outlier?
        #1.2, 1.75
        

### Now it is time for us to get averaged freq values over time
## Bu iki methoddan hangisini seçmeliyiz?  Değerler bayağı değişiyor gibi


In [ ]:
def divide_equal_bins(num_of_bins = 5, array= None):
    means = []
    pieces = np.array_split(array[0:2048], num_of_bins, axis =1)
    #5, 2048, 8
    
    for i in range(num_of_bins):
        ssum = 0
        for j in range((np.array(pieces).shape)[-1]):
            ssum = pieces[i][0:2048][j]
        
        means.append(ssum.mean())
        
        
    return np.array(means)

In [ ]:
divide_equal_bins(array = output_spectrogram(ms_[0][0].data, 256)[0])

In [ ]:
def divide_(num_of_bins = 5, array= None):
    means = []
    pieces = np.array_split(array[0:2048], num_of_bins, axis =1)
    #5, 2048, 8
    
    for i in range(num_of_bins):
        ssum = 0
        for j in range((np.array(pieces).shape)[-1]):
            mean = pieces[i][0:2048][j].mean()
            ssum += mean
        means.append(ssum)
        
        
    return np.array(means)

In [ ]:
divide_(array = output_spectrogram(ms_[0][0].data, 256)[0])

In [ ]:
def spec_values(array):
    x_, y_, z_ = [],[],[]
    for i in range(len(array[0])):
        _x = output_spectrogram(array[0][i].data, 256)[0]
        _y = output_spectrogram(array[1][i].data, 256)[0]
        _z = output_spectrogram(array[2][i].data, 256)[0]

        x_.append(divide_equal_bins(array = _x))
        y_.append(divide_equal_bins(array = _y))
        z_.append(divide_equal_bins(array = _z)) 
        
        
    return np.stack((x_, y_, z_), axis = 0)

In [ ]:
ms = spec_values(ms_)
mr = spec_values(mr_)
ar = spec_values(ar_)
ass = spec_values(as_)

In [ ]:
ar[0][0]

In [ ]:
ar[0][70]

In [ ]:
print(ms.shape)
print(mr.shape)
print(ar.shape)
print(ass.shape)

In [ ]:
mr[0][1]

In [ ]:
def dataframe_creator(array, disease = "None", df = pd.DataFrame()):
    values = []
    for i in range(len(array[1])):
        values.append([[array[0][i][0], array[0][i][1], array[0][i][2], array[0][i][3], array[0][i][4], 
                   array[1][i][0], array[1][i][1], array[1][i][2], array[1][i][3], array[1][i][4],
                   array[2][i][0], array[2][i][1], array[2][i][2], array[2][i][3], array[2][i][4],
                   disease]])


        medium = pd.DataFrame(values[i], columns =["spec_x1", "spec_x2", "spec_x3", "spec_x4", "spec_x5",
                                                "spec_y1", "spec_y2", "spec_y3", "spec_y4", "spec_y5",
                                                "spec_z1", "spec_z2", "spec_z3", "spec_z4", "spec_z5",
                                                   "disease"])
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [ ]:
df = dataframe_creator(mr, disease = "MR", 
                         df =pd.DataFrame(columns =["spec_x1", "spec_x2", "spec_x3", "spec_x4", "spec_x5",
                                                "spec_y1", "spec_y2", "spec_y3", "spec_y4", "spec_y5",
                                                "spec_z1", "spec_z2", "spec_z3", "spec_z4", "spec_z5",
                                                 "disease"]))

df = dataframe_creator(ar, disease = "AR", df = df )
df = dataframe_creator(ms, disease = "MS", df = df )
df = dataframe_creator(as_, disease = "AS", df = df )

df.reset_index(drop = True, inplace = True)

In [ ]:
df

In [ ]:
df.to_csv(dir_path + '/spectogram_coef.csv')